In [8]:
import pandas as pd
import numpy as np

In [9]:
def unik_values(data):#Menentukan unik values dari masing masing kolom dan mengreturn list
  bantu = []
  for i in data.columns:
    bantu.append(list(set(data[i])))
  data = dict(key=data.columns,nilai=bantu)
  df = pd.DataFrame(data)  
  return df

def pisah_predict(data,kolom,key):  #Memisahkan target menjadi dua list berbeda
  bantu = []
  if type(key) != list:
    for i in key:
      for j in i:
        bantu.append(data[data[kolom]==j])
    return bantu
  else:
    for i in key:
      bantu.append(data[data[kolom]==i])
    return bantu

def Peluang(sample,keseluruhan): #menentukan peluang
  return sample/keseluruhan

def entropy(Data):  #Menghitung Entropy 
  total = 0
  sigma = 0
  for i in range(len(Data)):
    total += len(Data[i])
  for i in Data:
    if len(i)==0:
      continue
    E = -Peluang(len(i),total)*np.log2(Peluang(len(i),total))
    sigma +=E 
  return sigma

def values(data,kolom,key,kolom_predict,predict_key): ##Fungsi memisahkan kolom sesuai variabel target 
  bantu = pisah_predict(data,kolom,key)
  bantu2 = []
  for i in range(len(bantu)):
    bantu2.append(pisah_predict(bantu[i],kolom_predict,predict_key))
  return bantu2  

def gain(DataFrame,kolom_values,key_values,kolom_predict,predict_key): #Menentukan Informasi gain dari values
  Target = pisah_predict(DataFrame,kolom_predict,predict_key)
  Entropy_Target = entropy(Target)
  bantu = pisah_predict(DataFrame,kolom_values,key_values)
  data_values = []
  for i in range(len(bantu)):
    data_values.append(pisah_predict(bantu[i],kolom_predict,predict_key))  
  sigma = 0
  for i in data_values:
    total = 0
    for j in range(len(i)):
      total+=len(i[j])
    sigma += (total/len(DataFrame))*entropy(i)
  return Entropy_Target-sigma

In [10]:
df = pd.read_excel('/content/drive/My Drive/FolderBaru/Decision Tree/Tabel Untuk Latihan Menghitung Entropy.xlsx')

In [11]:
unik = unik_values(df)
unik

,key,nilai
0,Handphone,"[H5, H8, H9, H14, H1, H2, H4, H3, H7, H13, H12..."
1,Baterai,"[Cukup, Lemah, Kuat]"
2,Kamera,"[Sedang, Tinggi, Rendah]"
3,Harga,"[Mahal, Murah, Sangat Mahal, Sangat Murah]"
4,Layak Rekomendasi,"[Tidak, YA]"


In [12]:
target = unik.loc[4]
target.nilai

['Tidak', 'YA']

In [13]:
atribut = unik.key.loc[1:3]
atribut

1    Baterai
2     Kamera
3      Harga
Name: key, dtype: object

In [14]:
class Tree():
  def __init__(self,df,attribute,target):
    self.uniq = unik_values(df)
    self.attribute = attribute
    self.target = target
    gain_dict = {}
    for i in self.attribute:
      gainn = gain(df,i,self.uniq.nilai[self.uniq['key']==i],self.target.key,self.target.nilai)
      gain_dict[i] = gainn
    self.root = max(gain_dict)
    self.simpul_root = []
    self.simpul = []
    self.cabang = []
    self.label = None
    self.daun = []
  def get_label(self,list_root):
    bantu = pisah_predict(df,self.root,list_root)
    anak = {}
    if type(list_root) != list:
      for i in list_root:
        for j in range(len(bantu)):
          anak[i[j]] = len(bantu[j])
    else:
      for i in range(len(bantu)):
        anak[list_root[i]] = len(bantu[i]) 
    self.label = anak
  
  def get_daun(self,atribute_c,target_c):
    anak = Tree(df,atribute_c,target_c)
    self.daun.append(anak.root)
    #return anak

In [15]:
def buildTree(df,tree=None): 
    Class = df.keys()[-1]   #To make the code generic, changing target variable class name
    
    #Here we build our decision tree

    #Get attribute with maximum information gain
    node = find_winner(df)
    
    #Get distinct value of that attribute e.g Salary is node and Low,Med and High are values
    attValue = np.unique(df[node])
    
    #Create an empty dictionary to create tree    
    if tree is None:                    
        tree={}
        tree[node] = {}
    
   #We make loop to construct a tree by calling this function recursively. 
    #In this we check if the subset is pure and stops if it is pure. 

    for value in attValue:
        
        subtable = get_subtable(df,node,value)
        clValue,counts = np.unique(subtable['Eat'],return_counts=True)                        
        
        if len(counts)==1:#Checking purity of subset
            tree[node][value] = clValue[0]                                                    
        else:        
            tree[node][value] = buildTree(subtable) #Calling the function recursively 
                   
    return tree
  

In [118]:
class TRee():
  def __init__(self,df,atribut,target):
    self.uniq = unik_values(df)
    self.df = df
    self.attribute = atribut
    self.target = target
    self.label = None
  def find_best_clasifier(self):
    gain_dict = {}
    for i in self.attribute:
      gainn = gain(df,i,self.uniq.nilai[self.uniq['key']==i],self.target.key,self.target.nilai)
      gain_dict[i] = gainn
    return max(gain_dict)
  def get_label(self,node,list_root):
    bantu = pisah_predict(df,node,list_root)
    anak = {}
    if type(list_root) != list:
      for i in list_root:
        for j in range(len(bantu)):
          anak[i[j]] = len(bantu[j])
    else:
      for i in range(len(bantu)):
        anak[list_root[i]] = len(bantu[i]) 
    self.label = anak



In [129]:
def get_subtable(df, node,value):
  return df[df[node] == value].reset_index(drop=True)
def ID3(df,atribut,target,tree=None):
  pohon = TRee(df,atribut,target)
  node = pohon.find_best_clasifier()
  if tree is None:                    
    tree={}
    tree[node] = {}
  try:
    pohon.get_label(node, pohon.uniq.nilai[pohon.uniq.key==node])
    for value in sorted(pohon.label,reverse=True):        
      subtable = get_subtable(df,node,value)
      clValue,counts = np.unique(subtable[df.keys()[-1]],return_counts=True)  
      if len(counts)==1:#Checking purity of subset
        tree[node][value] = clValue[0]                                                    
      else:    
        try:    
          for j in range(1, len(pohon.attribute)+1):
            if pohon.attribute[j] == node:
              pohon.attribute = pohon.attribute.drop(j).reset_index(drop=True)
              break
        except Exception :
          pohon.attribute = unik.key.loc[1:3]
          for j in range(1, len(pohon.attribute)+1):
            if pohon.attribute[j] == node:
              pohon.attribute = pohon.attribute.drop(j).reset_index(drop=True)
              break             
        tree[node][value] = ID3(subtable,pohon.attribute,target)
  except Exception:
    print("Error gan")
  return tree

In [130]:
ID3(df,atribut,target)

{'Kamera': {'Rendah': 'Tidak',
  'Sedang': {'Harga': {'Mahal': {'Baterai': {'Cukup': 'YA',
      'Kuat': 'YA',
      'Lemah': 'Tidak'}},
    'Murah': 'Tidak',
    'Sangat Mahal': 'YA'}},
  'Tinggi': {'Harga': {'Murah': 'YA',
    'Sangat Mahal': {'Baterai': {'Kuat': 'YA', 'Lemah': 'Tidak'}},
    'Sangat Murah': 'YA'}}}}

In [ ]:
def algoritma_id3(df,atribut,target):
  pohon = Tree(df,atribut,target)
  pohon.get_label(pohon.uniq.nilai[pohon.uniq.key==pohon.root])
  df_backup = df.copy(deep=True)
  if len(set(df_backup[target.key])) == 1 :
    pohon.simpul.append(set(df_backup[target.key])[0])
  if len(pohon.attribute) == 0:
    pohon.attribute = atribut
    algoritma_id3(df,pohon.attribute,target)
    #pohon.simpul.append(df_backup[target.key].mode()[0])
  else:
    for i in sorted(pohon.label,reverse=True):
      pohon.cabang.append(i)
      #print(pohon.simpul)
      df_backup = df_backup[df_backup[pohon.root]==i]
      print(pohon.root)
      print(pohon.label)
      print(pohon.cabang)
      print(pohon.daun)
      print(df_backup)
      #print(df_backup[target.key])
      if len(set(df_backup[target.key])) == 1 :
        pohon.daun.append(df_backup[target.key].mode()[0])
        pohon.attribute = atribut
        print(pohon.daun)
        continue
        
      if df_backup[target.key].empty:
        for i in df_backup[target.key].mode():
          pohon.daun.append(df_backup[target.key].mode())
        print(pohon.daun)
        pohon.attribute = atribut
      else:
        for j in range(1, len(pohon.attribute)+1):
          if pohon.attribute[j] == pohon.root:
            pohon.attribute = pohon.attribute.drop(j).reset_index(drop=True)
            break        
        algoritma_id3(df,pohon.attribute,target)
  tree =  {'Root':pohon.root,
          'cabang':pohon.cabang,
          'daun':pohon.daun} 
  return tree

In [ ]:
x = algoritma_id3(df,atribut,target)
x

Kamera
{'Tinggi': 6, 'Sedang': 5, 'Rendah': 3}
['Tinggi']
[]
   Handphone Baterai  Kamera         Harga Layak Rekomendasi
0         H1    Kuat  Tinggi  Sangat Murah                YA
1         H2    Kuat  Tinggi  Sangat Mahal                YA
4         H5   Cukup  Tinggi  Sangat Murah                YA
7         H8   Cukup  Tinggi         Murah                YA
9        H10   Lemah  Tinggi  Sangat Murah                YA
10       H11   Lemah  Tinggi  Sangat Mahal             Tidak
Harga
{'Sangat Mahal': 4, 'Murah': 2, 'Sangat Murah': 3, 'Mahal': 5}
['Sangat Murah']
[]
  Handphone Baterai  Kamera         Harga Layak Rekomendasi
0        H1    Kuat  Tinggi  Sangat Murah                YA
4        H5   Cukup  Tinggi  Sangat Murah                YA
9       H10   Lemah  Tinggi  Sangat Murah                YA
['YA']
Harga
{'Sangat Mahal': 4, 'Murah': 2, 'Sangat Murah': 3, 'Mahal': 5}
['Sangat Murah', 'Sangat Mahal']
['YA']
Empty DataFrame
Columns: [Handphone, Baterai, Kamera, Harga, Layak 

{'Root': 'Kamera', 'cabang': ['Tinggi', 'Sedang', 'Rendah'], 'daun': []}

In [ ]:
(df[target.key].mode()[0])

str

In [ ]:
target.key

key      Layak Rekomendasi
nilai          [Tidak, YA]
Name: 4, dtype: object

In [ ]:
df[(df['Kamera'] == 'Tinggi')&(df['Harga'] == 'Mahal')].empty

True

In [ ]:
x

{'Root': 'Kamera', 'cabang': ['Tinggi', 'Sedang', 'Rendah'], 'daun': []}

In [ ]:
y = {'a':10,
 'b':100,
 'c':5,
 'd':16}

In [ ]:
sorted(y.items(),key=lambda x: x[1],reverse=True)

[('b', 100), ('d', 16), ('a', 10), ('c', 5)]